## Measuring Investment Risk - WMT, META

## Calculating Covariance and Correlation

Consider a portfolio composed of *Walmart* and *Facebook*. Do you expect the returns of these companies to show high or low covariance? Or, could you guess what the correlation would be? Will it be closer to 0 or closer to 1? 

Begin by extracting data for Walmart and Facebook from the 1st of January 2018 until today.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
from pandas_datareader import data as wb

In [2]:
from iexfinance.stocks import Stock, get_historical_data

In [3]:
api_key = 'API_KEY'

In [4]:
start = datetime(2018, 1, 1)
end = datetime(2023, 3, 24)

In [5]:
tickers = ['WMT', 'META']
sec_data = pd.DataFrame()
for t in tickers:
    sec_data[t] = get_historical_data(t, start, end, output_format = 'pandas', token=api_key)['close']

In [6]:
sec_data = sec_data.astype('float64')
sec_data

,WMT,META
2018-01-02,98.59,181.42
2018-01-03,99.45,184.67
2018-01-04,99.54,184.33
2018-01-05,100.13,186.85
2018-01-08,101.61,188.28
...,...,...
2023-03-20,140.90,197.81
2023-03-21,140.42,202.16
2023-03-22,139.52,199.81
2023-03-23,140.65,204.28


In [7]:
sec_returns = np.log(sec_data / sec_data.shift(1))
sec_returns

,WMT,META
2018-01-02,NaN,NaN
2018-01-03,0.008685,0.017756
2018-01-04,0.000905,-0.001843
2018-01-05,0.005910,0.013579
2018-01-08,0.014673,0.007624
...,...,...
2023-03-20,0.010703,0.011184
2023-03-21,-0.003412,0.021752
2023-03-22,-0.006430,-0.011693
2023-03-23,0.008067,0.022125


In [8]:
sec_returns[['WMT', 'META']].mean() * 250

WMT     0.069097
META    0.024165
dtype: float64

In [9]:
sec_returns[['WMT', 'META']].std() * 250**0.5

WMT     0.231982
META    0.448498
dtype: float64

## Covariance and Correlation


\begin{eqnarray*}
Covariance Matrix: \  \   
\Sigma = \begin{bmatrix}
        \sigma_{1}^2 \ \sigma_{12} \ \dots \ \sigma_{1I} \\
        \sigma_{21} \ \sigma_{2}^2 \ \dots \ \sigma_{2I} \\
        \vdots \ \vdots \ \ddots \ \vdots \\
        \sigma_{I1} \ \sigma_{I2} \ \dots \ \sigma_{I}^2
    \end{bmatrix}
\end{eqnarray*}

In [10]:
sec_returns[['WMT', 'META']].var()

WMT     0.000215
META    0.000805
dtype: float64

In [11]:
sec_returns[['WMT', 'META']].var() * 250

WMT     0.053816
META    0.201150
dtype: float64

### Covariance matrix:

In [12]:
cov_matrix = sec_returns.cov()
cov_matrix

,WMT,META
WMT,0.000215,0.000094
META,0.000094,0.000805


In [13]:
cov_matrix = sec_returns.cov()*250
cov_matrix

,WMT,META
WMT,0.053816,0.023473
META,0.023473,0.201150


### Correlation matrix:

In [14]:
corr_matrix = sec_returns.corr()
corr_matrix

,WMT,META
WMT,1.000000,0.225607
META,0.225607,1.000000


## Calculating Portfolio Risk

Equal weigthing scheme:

In [16]:
weights = np.array([0.65, 0.35])

Portfolio Variance:

In [17]:
portfolio_var = np.dot(weights.T, np.dot(sec_returns.cov()*250,weights))
portfolio_var

0.05805824606888406

Portfolio Volatility:

In [18]:
portfolio_vol = np.dot(weights.T, np.dot(sec_returns.cov()*250,weights))**0.5
portfolio_vol

0.24095278804961784

In [19]:
print (str(round(portfolio_vol, 5) * 100) + '%')

24.095%


### Diversifiable Risk

In [22]:
WMT_var = sec_returns['WMT'].var()*250
WMT_var

0.05381567647610062

In [23]:
META_var = sec_returns['META'].var()*250
META_var

0.20115041133205586

In [24]:
dr = portfolio_var - (weights[0]**2 * WMT_var) - (weights[1]**2 * META_var)
dr

0.010680197369554709

In [25]:
print (str(round(dr*100, 3)) + '%')

1.068%


### Non Diversifiable

In [26]:
n_dr_1 = portfolio_var - dr
n_dr_1

0.04737804869932935

In [28]:
n_dr_2 = (weights[0]**2 * WMT_var) + (weights[1]**2 * META_var)
n_dr_2

0.04737804869932935

In [29]:
n_dr_1 == n_dr_2

True